<h1>Low-Rank Adaptation</h1>

<h2>Overview</h2>

In this post, I will delve into LoRA (Low-Rank Adaptation), a fine-tuning technique that has garnered significant attention recently. LoRA was introduced by Hu et al. in <a href=https://arxiv.org/abs/2106.09685> this paper</a> and represents a specialized approach within the broader spectrum of Parameter-Efficient Fine-Tuning (PEFT) techniques. Similar to my previous posts, I'll begin with an exploration of the intuition and theoretical underpinnings before progressing to practical implementation and technique. Initially, I'll provide a concise overview of PEFT, followed by a discussion on the fundamental principles of LoRA. To wrap up, I'll present an illustrative example, demonstrating the application of LoRA in code.

<h2>Parameter Efficient Finetuning (PEFT)</h2>

While fine-tuning a large foundational model may appear less daunting than training it from scratch, the process still involves loading the pre-trained model into memory and adjusting its extensive set of parameters, which demands substantial computational resources. Parameter Efficient Fine-Tuning Techniques, or PEFT for short, are designed to alleviate this burden by ensuring that only a small subset of parameters is updated during fine-tuning. This approach not only conserves computational power but also addresses another critical concern: catastrophic forgetting. Typically, when a model is fine-tuned using a specific dataset for a particular task, its ability to perform other tasks diminishes. This phenomenon is known as catastrophic forgetting. PEFT offers a solution to this problem by training a unique set of parameters for each task. During inference, these task-specific parameters can be swapped accordingly, allowing the model to maintain its proficiency across various tasks.

There are three types of PEFT:
<ul>
  <li> <b>Selective Fine-Tuning</b>: This approach involves selectively updating only a small fraction of the model's parameters while keeping the rest frozen. It aims to focus computational resources on parts of the model that are most relevant to the new task.</li>
  <li> <b>Reparameterization</b>: Reparameterization introduces new parameters that are functions of the original parameters. This technique allows the model to adapt to new tasks flexibly without directly modifying the foundational parameters.</li>
  <li> <b>Additive Fine-Tuning</b>: Additive Fine-Tuning involves adding small, task-specific modules or layers to the pre-trained model. These additional components, which are exclusively trained, enable the model to learn new capabilities while preserving its original knowledge.</li>
</ul>
Among these, LoRA falls under the category of reparameterization techniques. While each of these methods offers a unique approach to optimizing model performance with computational efficiency, discussing them in depth falls beyond the scope of this post.

<h2> Core Idea Behind LoRA</h2>

The core concept behind LoRA, as outlined in <a href='https://arxiv.org/abs/2012.13255'>this paper</a> by Aghajanyan et al., revolves around the intrinsic dimension of an objective function. This dimension represents the minimum number of parameters necessary to achieve a satisfactory solution, or a certain level of precision, for a given objective function. The authors suggest that pretrained models exhibit a notably low intrinsic dimension. This implies that there exists a method of reparameterization which, despite being of lower dimensionality, is as effective as utilizing the full set of parameters.

The paper further posits that the process of pretraining inherently reduces the intrinsic dimension. Moreover, it highlights an intriguing trend: as models grow larger, their intrinsic dimension tends to decrease proportionally, given a fixed number of pretraining updates. 

This phenomenon can be attributed to the increasing capacity and complexity of larger models, which allow them to represent and encapsulate more information effectively.

Larger models have more parameters and thus a greater potential to capture intricate patterns in the data. With a fixed number of pretraining updates, these models can efficiently organize their parameters to form more generalized representations of the data. That is after undergoing the same amount of pretraining, become more efficient in terms of the number of parameters required to achieve effective results.This efficient organization leads to a scenario where even though the total number of parameters is high, the number of parameters that need to be fine-tuned or adjusted to reach a satisfactory solution (the intrinsic dimension) is relatively low. 

The essence of LoRa, therefore, lies in leveraging this phenomenon to optimize model performance. By recognizing and harnessing the reduced intrinsic dimension of pretrained models, LoRa aims to achieve high levels of effectiveness without necessitating the full spectrum of parameters, thereby enhancing computational efficiency.
This approach maintains the model's performance while significantly reducing the computational overhead, making the process more efficient and resource-friendly. The intrinsic dimension's decrease in larger models, after a certain number of pretraining updates, underscores the models' ability to form more generalized and compact representations, a principle that LoRa ingeniously exploits to enhance model performance efficiently.

For a pretrained weight matrix $W_0 \in R^{d\times k}$, the updates through finetuning is like $\Delta W$; thus, the finetuned weight matrix is $W_0 + \Delta W$. With the above discussion about the intrinsic dimension, $\Delta W$ is not full-ranked. Thus, a low_rank decomposition is $W_0 + \Delta W = W_0 + B\times A$ where $B \in R^{d\times r}$, $A \in R^{r\times k}$, and $r < min(d,k)$. The fact that $r < min(d,k)$ means when finetuning, the number of parameters required in inetuning is much less that the pretrained model, in practice 15%-20%. 

How LoRA helps at inference time?

When deployed in production, we can compute and store $W = W_0 + B\times A$ and perform inference as usual. Both $W_0$ and $B\times A$ are in $R^{d\times k}$. When need to switch to another downstream task, we can recover $W_0$ by subtracting $B\times A$ and then adding a different $B^{'}\times A^{'}$, which is a quick operation with very little memory overhead. Moreover, regarding the storage, when finetuning for various downstream tasks, there is no need to finetune to have several models for each task. We can simply finetune by different $B\times A$ and we are storing only these $B$ and $A$ matrices according to each task, which are much lighter than the original pretrained model (think about LLMs with billions of parameters) then for different tasks, simply switch between these matrices. 